## Finetune chinese-llama/alpaca for couplet writing

In this notebook, we are going to use 100k Chinese couplet data to finetune the Chinese llama/alpaca LLM for couplet writing!

### 1. Load Packages

In [ ]:
# Prerequisites
#!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7 transformers[sentencepiece]==4.31.0 scipy tensorboard

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    GenerationConfig,
    HfArgumentParser,
    LlamaTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import warnings
warnings.filterwarnings('ignore')

In [ ]:
if torch.cuda.is_available():
    device = torch.device(0)
else:
    device = torch.device('cpu')

print(device)

### 2. Set up model and finetuning parameters

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "ziqingyang/chinese-alpaca-2-7b"

# The instruction dataset to use
dataset_name = "chenqile09/llama2-chinese-couplet-100k"

# Fine-tuned model name
new_model = "chinese-alpaca-2-LoRA-7B-couplet-100k"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 5000

# Log every X updates steps
logging_steps = 250

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

### 3. Load dataset, tokenizer, and base model from Hugging Face

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

In [ ]:
# Load tokenizer

tokenizer = LlamaTokenizer.from_pretrained(model_name)

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

print(compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# check GPU memory usage (optional)

!nvidia-smi

### 4. Load finetuning parameters

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="data",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

### 5. Finetune the model

In [ ]:
# Train model
trainer.train()

# if the model training stops in the middle, use the following to resume training from the checkpoint
# trainer.train(resume_from_checkpoint="./results/checkpoint-3525")

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

### 6. Merge model in FP16 and merge it with LoRA weights

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = LlamaTokenizer.from_pretrained(model_name)

In [ ]:
# need huggingface-cli login
!huggingface-cli login

In [ ]:
# push the model and tokenizer to hugging face
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

### 7. Test the new model

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [ ]:
# Set up the text generation config

generation_config = GenerationConfig(
    temperature=0.2,
    top_k=40,
    top_p=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.1,
    max_new_tokens=400
)

In [ ]:
# Set up system and input prompt template

COUPLET_SYSTEM_PROMPT = """你是一个写对联的助手，你会根据给出的上联来写下联。上联和下联的字数要一致，并且平仄要对应。"""

TEMPLATE = (
    "[INST] <<SYS>> {system_prompt} <</SYS>> {instruction} [/INST]"
)

def generate_prompt(instructions, system_prompt=COUPLET_SYSTEM_PROMPT):
    return TEMPLATE.format_map({'instruction': instruction,'system_prompt': system_prompt})

def generate_output_from_llm(model, tokenizer, generation_config, instruction, system_prompt=COUPLET_SYSTEM_PROMPT):

    input_text = generate_prompt(instruction, system_prompt)
    inputs = tokenizer(input_text, return_tensors="pt")  #add_special_tokens=False ?
    generation_output = model.generate(
        input_ids = inputs["input_ids"].to(device),
        attention_mask = inputs['attention_mask'].to(device),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        generation_config = generation_config
    )

    s = generation_output[0]
    output = tokenizer.decode(s, skip_special_tokens=True)
    response = output.split("[/INST]")[-1].strip()
    return response

In [ ]:
# set up a few example inputs

instructions = [
    "半衾幽梦香初散",
    "欲把心事付瑶琴",
    "半衾幽梦香初散",
    "一曲清音韵未央",
    "应是飞鸿踏泥雪",
    "云破月来花弄影",
    "别后相思空一水",
    "书中自有黄金屋",
    "花谢花飞花满天",
    "杨柳岸晓风残月",
    "上海自来水来自海上",
]

In [ ]:
# check out results for each input!

for instruction in instructions:

    response = generate_output_from_llm(
        model,
        tokenizer,
        generation_config,
        instruction,
        COUPLET_SYSTEM_PROMPT
    )

    print(instruction+','+response)